In [1]:
import os 
import sys
from  dataclasses import dataclass
import pathlib
from pathlib import Path

In [2]:
%pwd

'd:\\Projects\\Text Summerizer\\Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd 

'd:\\Projects\\Text Summerizer\\Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass()
class DataIngestionConfig:
    root_dir : Path
    source_url :str
    local_data_files : Path
    unzip_dir : Path
    

    





In [6]:
from textSummarizer.constants import CONFIG_FILE_PATH , PARAMS_FILE_PATH
from textSummarizer.utils.common import read_yaml
from textSummarizer.utils.common import create_directories

In [7]:
class ConfigManager(object):
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH ,
                 params_file_path = PARAMS_FILE_PATH): 
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifact_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_files=config.local_data_files,
            unzip_dir=config.unzip_dir


        )
        return data_ingestion_config

        



In [11]:
# components 
import os 
from urllib import request as request
import zipfile
from textSummarizer.logging import logger 
from textSummarizer.utils.common import get_size



In [18]:
class DataIngestion:
    def __init__(self , config:DataIngestionConfig):
        self.config = config

    def download_data(self):
        if not os.path.exists(self.config.local_data_files):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_files
                
            )
            logger.info(f'{filename}  download! with folder info \n{headers}')
        else : 
            logger.info(f"file already exist size of {get_size(Path(self.config.local_data_files))}")


    def extract_data(self):
        """
        Zip_file_path  : str 
        Extrtact the data from the zip file into the local data folder
        functions return None 
        """
        unzip_file_path = self.config.unzip_dir

        os.makedirs(unzip_file_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_files, 'r') as zip_ref:
            zip_ref.extractall(unzip_file_path)

In [20]:
# pipeline 

try : 
    config = ConfigManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_data()
except Exception as e: 
    raise  e

[2023-06-23 18:56:55,330: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-06-23 18:56:55,333: INFO: common: yaml file: params.yaml loaded successfully]
[2023-06-23 18:56:55,335: INFO: common: created directory at: artifacts]
[2023-06-23 18:56:55,337: INFO: common: created directory at: artifacts/data_ingestion]
[2023-06-23 18:56:55,340: INFO: 566250581: file already exist size of ~ 7718 KB]
